In [1]:
import os

In [2]:
%pwd

'c:\\Users\\46236358\\OneDrive - GLOBAL HITSS\\Documentos\\git personal\\end-to-end-ML-project-whit-MLflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\46236358\\OneDrive - GLOBAL HITSS\\Documentos\\git personal\\end-to-end-ML-project-whit-MLflow'

In [33]:
# entidad

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflw_uri: str

In [34]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [35]:
# configuration manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        SCHEMA_FILE_PATH = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflw_uri = "https://dagshub.com/fmafelipe/end-to-end-ML-project-whit-MLflow.mlflow"
        )

        return model_evaluation_config
    

In [36]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub

In [37]:
class ModelEvaluation:
    def __init__(self, config : ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        dagshub.init(repo_owner='fmafelipe', repo_name='end-to-end-ML-project-whit-MLflow', mlflow=True)

        mlflow.set_experiment('Wine_Experiment')
        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            mlflow.sklearn.log_model(model, "ModelWineQuality", registered_model_name="ElasticnetModel")


In [38]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2026-02-04 17:40:00,894: INFO: common]: yaml file: config\config.yaml loaded successfully]
[2026-02-04 17:40:00,898: INFO: common]: yaml file: params.yaml loaded successfully]
[2026-02-04 17:40:00,903: INFO: common]: yaml file: schema.yaml loaded successfully]
[2026-02-04 17:40:00,906: INFO: common]: created directory at: artifacts]
[2026-02-04 17:40:00,909: INFO: common]: created directory at: artifacts/model_evaluation]
[2026-02-04 17:40:02,404: INFO: _client]: HTTP Request: GET https://dagshub.com/api/v1/repos/fmafelipe/end-to-end-ML-project-whit-MLflow "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "fmafelipe/end-to-end-ML-project-whit-MLflow"

[2026-02-04 17:40:02,408: INFO: helpers]: Initialized MLflow to track repo "fmafelipe/end-to-end-ML-project-whit-MLflow"]


Repository fmafelipe/end-to-end-ML-project-whit-MLflow initialized!

[2026-02-04 17:40:02,408: INFO: helpers]: Repository fmafelipe/end-to-end-ML-project-whit-MLflow initialized!]
[2026-02-04 17:40:03,398: INFO: common]: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2026/02/04 17:40:11 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
